Check out https://docs.sciml.ai/ModelingToolkit/dev/

In [14]:
using ModelingToolkit
using LinearAlgebra

In [16]:
@parameters t  # Independent variables: unsere Variablen, die die Ableitung formen
@variables u, v
surf_coord = @variables x(u,v), y(u,v), z(u,v)  # surface coordinates parametrised by u,v

D = Symbolics.Differential(t)
Du = Symbolics.Differential(u)
Dv = Symbolics.Differential(v)
Duu = Du^2
Dvv = Dv^2


Differential(v) ∘ Differential(v)

first fundamental form https://en.wikipedia.org/wiki/First_fundamental_form

In [17]:
# The coefficients of the first fundamental form may be found by taking the dot product of the partial derivatives.
E = dot(Du(surf_coord), Du(surf_coord))
F = dot(Du(surf_coord), Dv(surf_coord))
G = dot(Dv(surf_coord), Dv(surf_coord))

# ... and their corresponding derivatives
Ev = Dv(E)
Eu = Du(E)
Fv = Dv(F)
Fu = Du(F)
Gv = Dv(G)
Gu = Du(G)

H = sqrt(E*G - F^2) 

LoadError: MethodError: no method matching iterate(::Term{Vector{Num}, Nothing})
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:872
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Integer[39m) at range.jl:872
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}} at dict.jl:712
[0m  ...

Christoffel symbols: parameters derived from the coefficients


In [4]:
l = (1/H^2)*(G*Eu - 2*F*Fu + F*Ev)
m = (1/H^2)*(G*Ev - F*Gu)
n = (1/H^2)*(2*G*Fv - G*Gu - F*Gv)
λ = (1/H^2)*(2*E*Fu - E*Ev + F*Eu)
μ = (1/H^2)*(E*Gu - F*Ev)
ν = (1/H^2)*(E*Gv - 2*F*Fv + F*Gu)

NaN + Inf*(Differential(u)(x(u, v))^2)*Differential(v)(Differential(v)(x(u, v))^2) + Inf*Differential(u)(Differential(v)(x(u, v))^2)*Differential(u)(x(u, v))*Differential(v)(x(u, v)) - Inf*Differential(v)(Differential(u)(x(u, v))*Differential(v)(x(u, v)))*Differential(u)(x(u, v))*Differential(v)(x(u, v))

In [23]:
# # velocity
# D(surf_coord) ~ Du(surf_coord)*D(u) + Dv(surf_coord)*D(v)

# # acceleration 
# D(D(surf_coord)) ~ Du(surf_coord)*D(D(u)) + Duu(surf_coord)*D(u)^2 + Dv(surf_coord)*D(D(v)) + Dvv(surf_coord)*D(v)^2 + 2*Dv(Du(surf_coord))*D(u)*D(v)

# velocity
D(x) ~ Du(x)*D(u) + Dv(x)*D(v)
D(y) ~ Du(y)*D(u) + Dv(y)*D(v)
D(z) ~ Du(z)*D(u) + Dv(z)*D(v)


# acceleration 
D(D(x)) ~ Du(x)*D(D(u)) + Duu(x)*D(u)^2 + Dv(x)*D(D(v)) + Dvv(x)*D(v)^2 + 2*Dv(Du(x))*D(u)*D(v)
D(D(y)) ~ Du(y)*D(D(u)) + Duu(y)*D(u)^2 + Dv(y)*D(D(v)) + Dvv(y)*D(v)^2 + 2*Dv(Du(y))*D(u)*D(v)
D(D(z)) ~ Du(z)*D(D(u)) + Duu(z)*D(u)^2 + Dv(z)*D(D(v)) + Dvv(z)*D(v)^2 + 2*Dv(Du(z))*D(u)*D(v)

Differential(t)(Differential(t)(z(u, v))) ~ (Differential(t)(u)^2)*Differential(u)(Differential(u)(z(u, v))) + (Differential(t)(v)^2)*Differential(v)(Differential(v)(z(u, v))) + Differential(t)(Differential(t)(u))*Differential(u)(z(u, v)) + Differential(t)(Differential(t)(v))*Differential(v)(z(u, v)) + 2Differential(t)(u)*Differential(t)(v)*Differential(v)(Differential(u)(z(u, v)))

The generalised **force vectors** $\psi$ and $\phi$ in the 2D mapping

In [6]:
force = [2,1,1]

component_1 = force[1]*Du(x) + force[2]*Du(y) + force[3]*Du(z)
component_2 = force[1]*Dv(x) + force[2]*Dv(y) + force[3]*Dv(z)

# ϕ = G*component_1 - F*component_2
# ψ = -F*component_1 + F*component_2

# TEMP: set the force to 0 
ϕ = 0
ψ = 0

0

Equations of motion of an unit mass particle  
NOTE: '~' indicates that it is an equation

In [7]:
eqs = [D(D(u)) ~ ϕ - l*D(u)^2 - 2*m*D(u)*D(v) - n*D(v)^2,
     D(D(v)) ~ ψ - λ*D(u)^2 - 2*μ*D(u)*D(v) - ν*D(v)^2] 
    
bcs = []

Any[]

Gaussian Peak

In [8]:
# x(u_,v_) = u
# y(u_,v_) = v
# a*exp(-(abs(x-x0)^2)/c)